# 욕설 탐지(lexicon 기반) 평가 노트

아래 코드는 `test/korean_swear_words.csv`의 욕설 사전을 이용해 간단한 규칙 기반(정규식) 탐지기를 만들고, `data/combined_abusive_shuffled_20k.csv`와 `data/nonabusive_merged_shuffled_sample20000.csv`에 대해 정확도(accuracy), 정밀도(precision), 재현율(recall), F1, 혼동행렬을 계산합니다. 전체 흐름은 다음과 같습니다.

1) 의존성 로드 및 경로 설정(모두 상대경로)
2) 욕설 사전(csv 단일 컬럼 `term`)을 읽어 정규식 패턴으로 컴파일
3) 텍스트에 패턴이 한 번이라도 매칭되면 욕설로 간주(1), 아니면 비욕설(0)
4) 두 데이터셋 개별/통합 평가 및 메트릭 집계
5) 예시 예측 몇 개를 출력해 탐지 결과를 빠르게 살펴보기


In [6]:
from pathlib import Path
import re

import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_colwidth", 120)


In [7]:
base_dir = Path(".")
lexicon_path = base_dir / "test/korean_swear_words.csv"
dataset_paths = {
    "abusive": base_dir / "data/combined_abusive_shuffled_20k.csv",
    "nonabusive": base_dir / "data/nonabusive_merged_shuffled_sample20000.csv",
}

# 욕설 사전 로드
lexicon_df = pd.read_csv(lexicon_path)
lexicon = (
    lexicon_df["term"]
    .dropna()
    .astype(str)
    .str.strip()
    .str.lower()
    .unique()
)

# 긴 단어 우선 매칭을 위해 길이 기준 정렬 후 정규식 컴파일
lexicon_sorted = sorted(lexicon, key=len, reverse=True)
pattern = re.compile("|".join(re.escape(term) for term in lexicon_sorted), flags=re.IGNORECASE)

print(f"욕설 사전 수: {len(lexicon_sorted)}개")


욕설 사전 수: 326개


In [8]:
print(f"욕설 사전 예시 (총 {len(lexicon_sorted)}개) — 상위 50개")
lexicon_preview = pd.DataFrame(lexicon_sorted, columns=["term"])
display(lexicon_preview.head(50))

욕설 사전 예시 (총 326개) — 상위 50개


,term
0,오스트랄로피테쿠스
1,밴댕이소갈딱지
2,몽근놈의자식
3,좆도아닌새끼
4,그지깽깽이
5,네다음xx
6,당나귀사촌
7,덜떨어지다
8,딸딸이부대
9,말하는가축


In [9]:
def predict_text_is_abusive(text: str) -> int:
    """패턴 매칭 결과를 0/1 라벨로 반환한다."""
    if not isinstance(text, str):
        return 0
    return 1 if pattern.search(text) else 0


def evaluate_dataset(df: pd.DataFrame, name: str):
    preds = df["text"].apply(predict_text_is_abusive)
    true = df["label"].astype(int)

    acc = accuracy_score(true, preds)
    prec = precision_score(true, preds, zero_division=0)
    rec = recall_score(true, preds, zero_division=0)
    f1 = f1_score(true, preds, zero_division=0)
    cm = confusion_matrix(true, preds)

    return {
        "dataset": name,
        "size": len(df),
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "cm": cm,
        "preds": preds,
    }


In [10]:
# 데이터셋 로드
abusive_df = pd.read_csv(dataset_paths["abusive"])
nonabusive_df = pd.read_csv(dataset_paths["nonabusive"])

# 평가
results = []
for name, df in [("abusive", abusive_df), ("nonabusive", nonabusive_df)]:
    results.append(evaluate_dataset(df, name))

all_df = pd.concat([abusive_df, nonabusive_df], ignore_index=True)
results.append(evaluate_dataset(all_df, "combined"))

# 결과 표 정리
summary_rows = []
for r in results:
    summary_rows.append(
        {
            "dataset": r["dataset"],
            "size": r["size"],
            "accuracy": round(r["accuracy"], 4),
            "precision": round(r["precision"], 4),
            "recall": round(r["recall"], 4),
            "f1": round(r["f1"], 4),
            "tn": int(r["cm"][0, 0]),
            "fp": int(r["cm"][0, 1]),
            "fn": int(r["cm"][1, 0]),
            "tp": int(r["cm"][1, 1]),
        }
    )

summary_df = pd.DataFrame(summary_rows)
display(summary_df)


,dataset,size,accuracy,precision,recall,f1,tn,fp,fn,tp
0,abusive,20000,0.4101,0.8121,0.4043,0.5398,1281,1601,10198,6920
1,nonabusive,20000,0.8704,0.0000,0.0000,0.0000,17409,2591,0,0
2,combined,40000,0.6402,0.6228,0.4043,0.4903,18690,4192,10198,6920


In [11]:
# 오탐/미탐 샘플 확인
abusive_preds = results[0]["preds"]
nonabusive_preds = results[1]["preds"]

false_negatives = abusive_df.loc[(abusive_df["label"] == 1) & (abusive_preds == 0), "text"].head(5)
false_positives = nonabusive_df.loc[(nonabusive_df["label"] == 0) & (nonabusive_preds == 1), "text"].head(5)
true_positives = abusive_df.loc[(abusive_df["label"] == 1) & (abusive_preds == 1), "text"].head(5)

print("[FN] 욕설을 놓친 샘플 (상위 5개)")
for i, t in enumerate(false_negatives, 1):
    print(f"{i}. {t}")

print("\n[FP] 욕설이 아닌데 탐지된 샘플 (상위 5개)")
for i, t in enumerate(false_positives, 1):
    print(f"{i}. {t}")

print("\n[TP] 욕설로 올바르게 탐지된 샘플 (상위 5개)")
for i, t in enumerate(true_positives, 1):
    print(f"{i}. {t}")


[FN] 욕설을 놓친 샘플 (상위 5개)
1. 자살하면 다음생 좋은데 갈꺼같냐ㅋㅋ?절때 못가지금수저로는 더더욱 못태어남100프로흙수저로 태어날듯
2. 완전 ᄒᄅᄂ~이네
3. 씨#발이 씨#발이 씨#발이
4. 주제도 오질라게 모르고 앉아있노 갈아탈 여자가 있을줄 아나보노
5. 이름이 부끄럽습니다

[FP] 욕설이 아닌데 탐지된 샘플 (상위 5개)
1. 개돼지가 그거 안 하면 도축 당하는 수밖에 없지
2. 응 느금마는 게임 안 해도 벌레임
3. 환경적인 문제도 있지만 개인 책임이 없지도 않음
4. 땅 샀는데, 그 부동산이 개발 가능하다고 속였대
5. 진짜 4050은 쓰레기가 맞다

[TP] 욕설로 올바르게 탐지된 샘플 (상위 5개)
1. ᄂᄀᄀᄈ 꺼져
2. 씹퇭아
3. 니좆이*다 니좆이*다 니좆이*다
4. 맞노 세상에는 어차피 수많은 논리들이 존재하기 때문에 맞고 그름이란 이론적으로 얼마든지 뒤집을수 있노 논리가 아니라 자기한테 이득가는 대로 가는게 진짜 맞는거노 내가 정한것도 아닌 도덕적 잣대 주입받고 그거에 맞춰살려고 끙끙 대고하는거는 개개개돼지 맞노
5. 너 진짜 게년 같다
